#Government Scheme Finder

###Use cases: - A student searches for educational scholarships. - A senior citizen looks for pension schemes.


In [1]:
import pandas as pd
from google.colab import files

In [2]:
df = pd.DataFrame({
    "scheme_name": ["Pradhan Mantri Jan-Dhan Yojana", "National Pension Scheme", "Sukanya Samriddhi Yojana", "Pradhan Mantri Ujjwala Yojana"],
    "eligibility_criteria": ["Age: 18-65, Financial inclusion", "Age: 18-60, Retirement benefits", "Age: 10-18, Education benefits", "Age: 18-60, LPG connection"],
    "benefits": ["Financial inclusion", "Retirement benefits", "Education benefits", "LPG connection"],
    "application_process": ["Online application", "Offline application", "Offline application", "Online application"]
})

df2 = pd.DataFrame({
    "age": [20, 30, 40, 50],
    "gender": ["Male", "Female", "Male", "Female"],
    "income": [20000, 30000, 40000, 50000],
    "education": ["Graduate", "Postgraduate", "High School", "Postgraduate"],
    "occupation": ["Student", "Working Professional", "Self Employed", "Retired"]
})

In [3]:
df.to_csv('government_schemes.csv', index=False)
df2.to_csv('demographic_data.csv', index=False)

In [4]:
files.download('government_schemes.csv')
files.download('demographic_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
!pip install requests
import requests

In [6]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import groq

# Load the dataset
schemes_df = pd.read_csv('/content/government_schemes.csv')
demographic_df = pd.read_csv('/content/demographic_data.csv')


schemes_df.head()



,scheme_name,eligibility_criteria,benefits,application_process
0,Pradhan Mantri Jan-Dhan Yojana,"Age: 18-65, Financial inclusion",Financial inclusion,Online application
1,National Pension Scheme,"Age: 18-60, Retirement benefits",Retirement benefits,Offline application
2,Sukanya Samriddhi Yojana,"Age: 10-18, Education benefits",Education benefits,Offline application
3,Pradhan Mantri Ujjwala Yojana,"Age: 18-60, LPG connection",LPG connection,Online application


In [8]:
demographic_df.head()

,age,gender,income,education,occupation
0,20,Male,20000,Graduate,Student
1,30,Female,30000,Postgraduate,Working Professional
2,40,Male,40000,High School,Self Employed
3,50,Female,50000,Postgraduate,Retired


In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import groq

# Load the dataset
schemes_df = pd.read_csv('/content/government_schemes.csv')
demographic_df = pd.read_csv('/content/demographic_data.csv')

# Define a function to preprocess the data
def preprocess_data(schemes_df, demographic_df):
    # Convert the schemes data to lowercase
    schemes_df['scheme_name'] = schemes_df['scheme_name'].str.lower()
    schemes_df['eligibility_criteria'] = schemes_df['eligibility_criteria'].str.lower()

    # Convert the demographic data to lowercase
    demographic_df['age'] = demographic_df['age'].astype(str).str.lower()
    demographic_df['gender'] = demographic_df['gender'].str.lower()
    demographic_df['income'] = demographic_df['income'].astype(str).str.lower()
    demographic_df['education'] = demographic_df['education'].str.lower()
    demographic_df['occupation'] = demographic_df['occupation'].str.lower()

    return schemes_df, demographic_df

# Preprocess the data
schemes_df, demographic_df = preprocess_data(schemes_df, demographic_df)

# Define a function to find eligible schemes
def find_eligible_schemes(user_input, schemes_df, demographic_df):
# Use TF-IDF to convert the user input and eligibility criteria to vectors
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer to both user input and eligibility criteria
    all_text = [user_input] + schemes_df['eligibility_criteria'].tolist()
    vectorizer.fit(all_text)

    # Transform user input and eligibility criteria into vectors
    user_input_vector = vectorizer.transform([user_input])
    eligibility_criteria_vectors = vectorizer.transform(schemes_df['eligibility_criteria'])

    # Use cosine similarity to find the most similar schemes
    similarities = cosine_similarity(user_input_vector, eligibility_criteria_vectors)

    # Get the index of the most similar scheme
    most_similar_index = np.argmax(similarities)

    # Get the most similar scheme
    most_similar_scheme = schemes_df.iloc[[most_similar_index]]

    # Filter the schemes based on the user's demographic information
    # ... (Existing filtering logic) ...

    return most_similar_scheme # Return the most similar scheme

    # Filter the schemes based on the user's demographic information
    eligible_schemes = most_similar_schemes[most_similar_schemes['eligibility_criteria'].str.contains(demographic_df['age'])]
    eligible_schemes = eligible_schemes[eligible_schemes['eligibility_criteria'].str.contains(demographic_df['gender'])]
    eligible_schemes = eligible_schemes[eligible_schemes['eligibility_criteria'].str.contains(demographic_df['income'])]
    eligible_schemes = eligible_schemes[eligible_schemes['eligibility_criteria'].str.contains(demographic_df['education'])]
    eligible_schemes = eligible_schemes[eligible_schemes['eligibility_criteria'].str.contains(demographic_df['occupation'])]

    return eligible_schemes

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from transformers import pipeline # Not needed in this context
# Define a function to generate output using LLM
from groq import Groq
def generate_output(eligible_schemes, api_key):
    # Use Groq API to generate output
    client = Groq(api_key=api_key)

    # Use client.chat.completions.create() to interact with Groq models
    prompt = f"Generate a description for the following government schemes: {eligible_schemes['scheme_name'].tolist()}"
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        model="llama3-8b-8192",  # or other appropriate model
    )
    output = response.choices[0].message.content  # Extract the generated text from the response

    return output

# Test the application
#I am a student looking for educational scholarships."
user_input = "I am a single mother looking for financial assistance for my child's education."
eligible_schemes = find_eligible_schemes(user_input, schemes_df, demographic_df)
api_key = "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"
output = generate_output(eligible_schemes, api_key)

print(output)

Here is a description for the Sukanya Samriddhi Yojana:

**Sukanya Samriddhi Yojana (SSY)**

The Sukanya Samriddhi Yojana (SSY) is a government-sponsored savings scheme designed to promote the education and empowerment of girl children. Launched in 2015, the SSY is a part of the Beti Bachao, Beti Padhao (Save the Girl Child, Educate the Girl Child) campaign. The scheme aims to enable parents and guardians to save for their daughter's education and marriage by providing a guaranteed rate of return and tax benefits.

**Key Features:**

1. **Eligibility:** The scheme is open to all Indian citizens, regardless of their socio-economic background. A girl child can be opened an account under the SSY scheme from the age of 10 years.
2. **Account Opening:** The account can be opened by a parent or legal guardian in the name of the girl child.
3. **Deposit:** The scheme allows for regular deposits of a minimum of Rs. 1,000 and a maximum of Rs. 1.5 lakh per year.
4. **Interest Rate:** The scheme 